In [12]:
from pandas import DataFrame
import pickle
import networkx as nx
import pandas as pd

pd.options.mode.copy_on_write = True

In [13]:
def check_status(ban_status: bool):
        if ban_status == True:
            return 'Cheater'
        return 'Legit'

In [14]:
def get_privacy_setting(privacy_setting_num: int):
        if privacy_setting_num == 3:
            privacy_setting = 'Public'
        elif privacy_setting_num == 1:
            privacy_setting = 'Private'
        else:
            privacy_setting = 'Friends-Only or Other'
        return privacy_setting

In [15]:
def generate_vac_graph_only_processed():
    G = nx.Graph()
    with open("data/bloomData.pkl", "rb") as dataPickle:
        df_full_data: DataFrame = pickle.load(dataPickle, fix_imports=True)
    
    processedIDs = set[str]([id for id in df_full_data.index])
    
    for _, row in df_full_data.iterrows():
        current_id = row['steamid']
        vac_banned = row['VACBanned']
        privacy_num = row['communityvisibilitystate']
        
        status = check_status(vac_banned)
        privacy = get_privacy_setting(privacy_num)
        
        G.add_node(current_id, status = status, privacy = privacy)
        

        
        for friend in row["friendsList"]:
            if friend['steamid'] in processedIDs:
                
                friend_ban_status = df_full_data[df_full_data['steamid'] == friend['steamid']]['VACBanned'].iloc[0]
                friend_status = check_status(friend_ban_status)
                friend_privacy_num = df_full_data[df_full_data['steamid'] == friend['steamid']]['communityvisibilitystate'].iloc[0]
                friend_privacy = get_privacy_setting(friend_privacy_num)
                
                G.add_node(friend['steamid'], status = friend_status, privacy = friend_privacy)
                G.add_edge(current_id, friend['steamid'], edge_type='friend')
    print('Done')
    nx.write_gexf(G, "./data/tennative_graph_only_processed_new.gexf")
    return G


In [16]:
def generate_vac_graph_all():
    G = nx.Graph()
    with open("data/bloomData.pkl", "rb") as dataPickle:
        df_full_data: DataFrame = pickle.load(dataPickle, fix_imports=True)
    
    processedIDs = set[str]([id for id in df_full_data.index])
    
    for _, row in df_full_data.iloc[:10000, :].iterrows():
        current_id = row['steamid']
        vac_banned = row['VACBanned']
        privacy_num = row['communityvisibilitystate']
        
        status = check_status(vac_banned)
        privacy = get_privacy_setting(privacy_num)
        
        G.add_node(current_id, status = status, privacy = privacy, process_status = 'Processed')
        

        
        for friend in row["friendsList"]:
            if friend['steamid'] in processedIDs:
                
                friend_ban_status = df_full_data[df_full_data['steamid'] == friend['steamid']]['VACBanned'].iloc[0]
                friend_status = check_status(friend_ban_status)
                friend_privacy_num = df_full_data[df_full_data['steamid'] == friend['steamid']]['communityvisibilitystate'].iloc[0]
                friend_privacy = get_privacy_setting(friend_privacy_num)
                
                G.add_node(friend['steamid'], status = friend_status, privacy = friend_privacy, process_status = 'Processed')
                G.add_edge(current_id, friend['steamid'], edge_type='friend')
            else:
                G.add_node(friend['steamid'], process_status = 'Un-Processed')
                G.add_edge(current_id, friend['steamid'], edge_type='friend')
    print('Done')
    nx.write_gexf(G, "./data/tennative_graph_all_new.gexf")
    return G

In [17]:
def generate_vac_graph_only_processed_only_full_friends():
    G = nx.Graph()
    with open("data/bloomData.pkl", "rb") as dataPickle:
        df_full_data: DataFrame = pickle.load(dataPickle, fix_imports=True)
    
    processedIDs = set[str]([id for id in df_full_data.index])
    
    for _, row in df_full_data.iterrows():
        current_id = row['steamid']
        vac_banned = row['VACBanned']
        privacy_num = row['communityvisibilitystate']
        
        status = check_status(vac_banned)
        privacy = get_privacy_setting(privacy_num)
        
        # G.add_node(current_id, status = status, privacy = privacy)
        

        friend_list = []
        process_status = "Fully"
        for friend in row["friendsList"]:
            if friend['steamid'] in processedIDs:
                
                friend_list.append(friend['steamid'])
                
                # friend_ban_status = df_full_data[df_full_data['steamid'] == friend['steamid']]['VACBanned'].iloc[0]
                # friend_status = check_status(friend_ban_status)
                # friend_privacy_num = df_full_data[df_full_data['steamid'] == friend['steamid']]['communityvisibilitystate'].iloc[0]
                # friend_privacy = get_privacy_setting(friend_privacy_num)
                
                # G.add_node(friend['steamid'], status = friend_status, privacy = friend_privacy)
                # G.add_edge(current_id, friend['steamid'], edge_type='friend')
            else:
                process_status = "Partial"
                break
        if process_status == "Fully":
            G.add_node(current_id, status = status, privacy = privacy, process_status=process_status)
            for friend in friend_list:
                
                friend_ban_status = df_full_data[df_full_data['steamid'] == friend]['VACBanned'].iloc[0]
                friend_status = check_status(friend_ban_status)
                friend_privacy_num = df_full_data[df_full_data['steamid'] == friend]['communityvisibilitystate'].iloc[0]
                friend_privacy = get_privacy_setting(friend_privacy_num)
                
                G.add_node(friend, status = friend_status, privacy = friend_privacy)
                G.add_edge(current_id, friend, edge_type='friend')
        else:
            G.add_node(current_id, status = status, privacy = privacy, process_status=process_status)
                
    print('Done')
    nx.write_gexf(G, "./data/tennative_graph_only_processed_new_full_friends.gexf")
    return G

In [18]:
def generate_vac_graph_processed_with_games_directed():
    G = nx.DiGraph()
    with open("data/bloomData.pkl", "rb") as dataPickle:
        df_full_data: DataFrame = pickle.load(dataPickle, fix_imports=True)
    
    processedIDs = set[str]([id for id in df_full_data.index])
    
    for _, row in df_full_data.iterrows():
        current_id = row['steamid']
        vac_banned = row['VACBanned']
        privacy_num = row['communityvisibilitystate']
        
        status = check_status(vac_banned)
        privacy = get_privacy_setting(privacy_num)
        
        G.add_node(current_id, type = 'User', status = status, privacy = privacy)
        if isinstance(row['ownedGames'], list):
            for game in row['ownedGames']:
                G.add_node(game['name'], type = 'Game')
                G.add_edge(current_id, game['name'])
        
        for friend in row["friendsList"]:
            if friend['steamid'] in processedIDs:
                
                friend_ban_status = df_full_data[df_full_data['steamid'] == friend['steamid']]['VACBanned'].iloc[0]
                friend_status = check_status(friend_ban_status)
                
                friend_privacy_num = df_full_data[df_full_data['steamid'] == friend['steamid']]['communityvisibilitystate'].iloc[0]
                friend_privacy = get_privacy_setting(friend_privacy_num)
                
                G.add_node(friend['steamid'], status = friend_status, privacy = friend_privacy)
                
                friend_games = df_full_data[df_full_data['steamid'] == friend['steamid']]['ownedGames'].iloc[0]
                if isinstance(friend_games, list):
                    for game in friend_games:
                        G.add_node(game['name'], type = 'Game')
                        G.add_edge(friend['steamid'], game['name'])
                
                if friend_ban_status == True and vac_banned == True:
                    
                    friend_last_banned = df_full_data[df_full_data['steamid'] == friend['steamid']]['DaysSinceLastBan'].iloc[0]
                    current_last_banned = row['DaysSinceLastBan']
                    
                    if current_last_banned < friend_last_banned:
                        G.add_edge(current_id, friend['steamid'], edge_type='friend')
                    else:
                        G.add_edge(friend['steamid'], current_id, edge_type='friend')
                else:
                    G.add_edge(current_id, friend['steamid'], edge_type='friend')
                    G.add_edge(friend['steamid'], current_id, edge_type='friend')
                        
                    
    print('Done')
    nx.write_gexf(G, "./data/tennative_graph_only_processed_with_games_directed_new.gexf")
    return G

In [19]:
# G = generate_vac_graph_processed_with_games_directed()

In [20]:
# G = generate_vac_graph_only_processed()

In [21]:
# G = generate_vac_graph_only_processed_only_full_friends()

In [ ]:
G = generate_vac_graph_all()